# Duplicate an observing script from N540 $\rightarrow$ N708

In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import pandas as pd
from astropy import coordinates
from astropy import units as u
from skipper import observe, planner

In [3]:
import sys
sys.path.append('../scripts/')

In [4]:
import sigjson_f2022b
import our_pointings

In [5]:
obsdates = sigjson_f2022b.obsdates

In [6]:
obskeys = [ f'{x[0]:02d}-{x[1]:02d}-{x[2]:02d}' for x in obsdates[:,:3]]

In [7]:
hcat, ocat = our_pointings.load_fallfields ()

In [47]:
%%timeit
# \\ load N708 pointings
halpha_locations = coordinates.SkyCoord( hcat['RA'], hcat['dec'], unit=('deg','deg'))


23.7 ms ± 3.03 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
# load nightly scripts from OIII
idx=0
year,month,day,_ = sigjson_f2022b.obsdates[sigjson_f2022b.obsfilters=='N540', :-1][idx]
datekey = f'{year}{month:02d}{day:02d}'

In [23]:
odir = f'../json/{datekey}/'

In [25]:
json_scripts = glob.glob(odir+'*json')

In [27]:
script_index=0



In [46]:
json_scripts[0]

'../json/20220920/20220921_09.json'

In [42]:

    jscript = pd.read_json(json_scripts[script_index])
    planned_oiii_obscoords = coordinates.SkyCoord( jscript['RA'], 
                                                   jscript['dec'], 
                                                   unit=('deg','deg'))
    # \\ match via coordinates
    matchids, sep, _ = planned_oiii_obscoords.match_to_catalog_sky ( halpha_locations )
    matches = sep < 0.05*u.arcsec

    assert matches.sum() == len(planned_oiii_obscoords)

    match_indices = hcat.index[matchids[matches]]

    dummy_obscat = observe.ObsCatalog ( propid='2020B-0288', seqid='2022B')

    dummy_obscat.to_json ( hcat.reindex(match_indices), fp='/Users/kadofong/Downloads/test.json', insert_checksky_exposures=False)